## Задание 1.
Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео.

Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 156.

Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10%.

Средний трафик 20000 человек в день.

Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20%.

In [55]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

In [56]:
def proportion_es(p1: float, p2: float):    
    return abs(proportion_effectsize(p1, p2))

def proportion_es_alt(conv1: float, conv2:float, p1:float, p2: float):
    
    return 2 * asin(np.sqrt(conv1/nobs1)) - 2 * asin(np.sqrt(conv2/nobs2))

def continuous_es(mean_control: Union[float, int],
                       std_control: Union[float, int],
                       mean_test: Union[float, int],
                       std_test: Union[float, int]):
    
    return abs(effectsize_smd(mean_control,
                              std_control,
                              1e4,
                              mean_test,
                              std_test,
                              1e4)[0])

def continuous_es_alt(mean_control: Union[float, int],
                           std_control: Union[float, int],
                           mean_test: Union[float, int],
                           std_test: Union[float, int]):
    
    effect_size = (mean_test - mean_control) / ((std_control**2 + std_test**2) / 2) ** 0.5
    return effect_size
    
def sample_size_continuous(effect_size: float,
                     alpha: float = .05,
                     beta: float = .2,
                     ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio)
    return int(n * 2)

def sample_size_proportion(effect_size: float,
                     alpha: float = .05,
                     beta: float = .2,
                     ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio)
    return int(n * 2)

In [57]:
ts_control, ts_test = 25, 27.5
std_control, std_test = 156, 156 * 1.15

es_cont = continuous_es(ts_control, std_control, ts_test, std_test)
es_cont_alt = continuous_es_alt(ts_control, std_control, ts_test, std_test)
es_cont, es_cont_alt

(0.014870865944557932, 0.014871423685692022)

In [58]:
sample_size_continuous(es_cont), sample_size_continuous(es_cont_alt)

(141971, 141960)

In [59]:
DAU = 20000
res = sample_size_continuous(es_cont) / DAU
print(f"Необходимо собирать данные в течении {res} дней.")

Необходимо собирать данные в течении 7.09855 дней.


## Задание 2.
Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта.

Одна из основных метрик bounce rate в GA = 40%.

Мы предполагаем, что при оптимизации сайта она изменится минимум на 20%.

Средний трафик 4000 человек в день.

Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%.

In [60]:
BR_1, BR_2 = 0.40, 0.32

es_prop = proportion_es(BR_1, BR_2)
es_prop

0.16690997264630925

In [61]:
sample_size_proportion(es_prop)

1126

In [62]:
DAU = 4000
res = sample_size_proportion(es_prop) / DAU
print(f"Необходимо собирать данные в течении {res} дней.")

Необходимо собирать данные в течении 0.2815 дней.
